In [9]:
try:
    import google.colab
    on_colab = True

    !gdown --id 1egJIl0p1rw3FAaeumFQ5ydYhZCKsrt67 -O patterns.zip
    !unzip patterns.zip -d ./patterns

    !gdown --id 1hYjHNM6333H8-53bF4jimIVRfhM8JxML -O images.zip
    !unzip images.zip -d ./images

except ImportError:
    on_colab = False

In [10]:
if on_colab:
    !git clone https://github.com/hamidrezafahimi/depth_pattern_analysis.git

In [18]:
import os
import sys
if on_colab:
    sys.path.append('/content/depth_pattern_analysis/train')
    imgdir = '/content/images'
    patdir = '/content/patterns'
    chdir = '/content/checkpoints'
    os.makedirs("/content/output", exist_ok=True)  # Creates the folder if it doesn't exist
    outputdir = "/content/output/"
else:
    sys.path.append('../train')
    imgdir = '../data/images'
    patdir = '../data/patterns'
    chdir = '../data/checkpoints'
    outputdir = "../data/output/"

from helper1 import *
model, dataloader = create_model(img_directory=imgdir, pat_directory=patdir)

In [12]:
train_model(model, dataloader, epochs=20, alpha=0.01, lr=1e-4, device='cuda', checkpoint_dir=chdir)

Epoch [1/20], Loss: 0.0320
Epoch [2/20], Loss: 0.0211
Epoch [3/20], Loss: 0.0169
Epoch [4/20], Loss: 0.0149
Epoch [5/20], Loss: 0.0147
Checkpoint saved at ./checkpoints/checkpoint_epoch_5.pth
Epoch [6/20], Loss: 0.0137
Epoch [7/20], Loss: 0.0139
Epoch [8/20], Loss: 0.0146
Epoch [9/20], Loss: 0.0133
Epoch [10/20], Loss: 0.0133
Checkpoint saved at ./checkpoints/checkpoint_epoch_10.pth
Epoch [11/20], Loss: 0.0131
Epoch [12/20], Loss: 0.0131
Epoch [13/20], Loss: 0.0126
Epoch [14/20], Loss: 0.0130
Epoch [15/20], Loss: 0.0130
Checkpoint saved at ./checkpoints/checkpoint_epoch_15.pth
Epoch [16/20], Loss: 0.0123
Epoch [17/20], Loss: 0.0123
Epoch [18/20], Loss: 0.0126
Epoch [19/20], Loss: 0.0124
Epoch [20/20], Loss: 0.0126
Checkpoint saved at ./checkpoints/checkpoint_epoch_20.pth


In [20]:
# Inference on new data
import random
import numpy as np
from PIL import Image

# Set device (GPU if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load model
e_model = EncoderDecoder(in_channels=2).to(device)  # Ensure model is moved to device

checkpoint = torch.load(chdir + "/checkpoint_epoch_20.pth", map_location="cuda" if torch.cuda.is_available() else "cpu")
e_model.load_state_dict(checkpoint["model_state_dict"])

# Set model to evaluation mode
e_model.eval()

# Run inference
with torch.no_grad():
    for x_test, p_test in dataloader:
        x_test = x_test.cuda()
        p_test = p_test.cuda()
        m_pred = e_model(x_test, p_test)  # shape (B,1,H,W)

        # Pick a random index from the batch
        rand_idx = random.randint(0, x_test.shape[0] - 1)

        # Extract the randomly chosen sample
        x_sample = x_test[rand_idx].cpu().numpy().squeeze()
        p_sample = p_test[rand_idx].cpu().numpy().squeeze()
        m_sample = m_pred[rand_idx].cpu().numpy().squeeze()

        # Convert arrays to grayscale images and save
        Image.fromarray((x_sample * 255).astype(np.uint8)).save(outputdir + "x_input.jpg")
        Image.fromarray((p_sample * 255).astype(np.uint8)).save(outputdir + "p_pattern.jpg")
        Image.fromarray((m_sample * 255).astype(np.uint8)).save(outputdir + "m_predicted.jpg")

        print(f"Saved: x_input.jpg, p_pattern.jpg, m_predicted.jpg (random index {rand_idx})")
        break  # Save only one random sample, then exit loop

Saved: x_input.jpg, p_pattern.jpg, m_predicted.jpg (random index 2)
